##Install Important Libraries

In [ ]:
!pip install bs4

Link
https://colab.research.google.com/drive/1FZibyCe7hAkZEi-FUGmh3lXLC5qutt1Y#scrollTo=cgEKaZTXXGC8

In [ ]:
!pip install jmd_imagescraper

In [ ]:
#export

# scraping
from pathlib import Path
from typing import Union
from enum import Enum
import re
import requests
import json
import time
from bs4 import BeautifulSoup
import uuid

# other
from PIL import Image as PImage
from IPython.display import display
import pandas as pd
from fastprogress.fastprogress import progress_bar

In [ ]:
from google import colab
colab.drive.mount('/content/gdrive')

Mounted at /content/gdrive


##Download UNPLASH images

In [ ]:
from pathlib import Path
lik = Path().cwd()/"https://unsplash.com/"

from jmd_imagescraper.core import * # dont't worry, it's designed to work with import *

duckduckgo_search(lik, "Beach", "Beach", max_results=400)
duckduckgo_search(lik, "People", "People", max_results=1000)
duckduckgo_search(lik, "Aerial Images", "Aerial Images", max_results=1000)
duckduckgo_search(lik, "Building", "Building", max_results=5000)
duckduckgo_search(lik, "Photography", "Photography", max_results=5000)
duckduckgo_search(lik, "Animals", "Animals", max_results=5000)

Duckduckgo search: Beach


Duckduckgo search: People


Duckduckgo search: Aerial Images


Duckduckgo search: Building


Duckduckgo search: Photography


Exception occured while retrieving https://tse3.mm.bing.net/th?id=OIP.xO-Ml6PYpiFwKkSdWXMlfwHaHa&pid=Api
Duckduckgo search: Animals


[PosixPath('/content/https:/unsplash.com/Animals/001_92f3f599.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/002_39493fe8.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/003_814f4ef1.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/004_bf2a8859.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/005_ae58f86e.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/006_64d7b878.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/007_91c58147.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/008_7a92f520.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/009_6e1831a7.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/010_d8a3f2f0.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/011_a6a46cca.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/012_9d7c0070.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/013_370e9016.jpg'),
 PosixPath('/content/https:/unsplash.com/Animals/014_7d6550eb.jpg'),
 PosixPath('/content/https:/unspla

##Defining The Path

In [ ]:
#export
def rmtree(path: Union[str, Path]):
    '''Recursively delete a directory tree'''
    path = Path(path); assert path.is_dir()
    for p in reversed(list(path.glob('**/*'))):
        if p.is_file():  p.unlink()
        elif p.is_dir(): p.rmdir()
    path.rmdir()

In [ ]:

def download_urls(path: Union[str, Path], links: list, uuid_names: bool=True) -> list:
  '''Downloads urls to the given path. Returns a list of Path objects for files downloaded to disc.'''
  if(len(links) == 0):
    print("Nothing to download!"); return

  path = Path(path)
  path.mkdir(parents=True, exist_ok=True)

  print("Downloading results into", path)
  pbar = progress_bar(links)
  pbar.comment = 'Images downloaded'

  i = 1
  mk_uniq = lambda : '_' + str(uuid.uuid4())[:8] if uuid_names else ''
  mk_fp = lambda x: path/(str(x).zfill(3) + mk_uniq() + ".jpg")
  is_file = lambda x: len(list(path.glob(str(x).zfill(3) + '*.jpg'))) > 0
    
  while is_file(i): i += 1 # don't overwrite previous searches
  
  results = []
    
  for link in pbar:
      try:
        resp = requests.get(link)
        fp = mk_fp(i)
        fp.write_bytes(resp.content)

        try:
          img = PImage.open(fp)
          img.verify()
          img.close()
          results.append(Path(fp))
        except Exception as e:
          # print(e)
          print(fp, "is invalid")
          fp.unlink()
      except Exception as e:
        # print(e)
        print("Exception occured while retrieving", link)
        
      i += 1

  return results

In [ ]:
params = {
    "max_results": 10000,
     
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.All,
    
}

links = duckduckgo_scrape_urls("Unplash", **params)
links

['https://tse4.mm.bing.net/th?id=OIP._929W-twSt3yNx_Hd0gBnQHaE7&pid=Api',
 'https://tse4.mm.bing.net/th?id=OIP.NWKhJXtP1Mh57tPX3CprQQHaFj&pid=Api',
 'https://tse2.mm.bing.net/th?id=OIP.H5T5wthEShwVZGR3ME4gCgHaE8&pid=Api',
 'https://tse2.mm.bing.net/th?id=OIP.wVls1ApDMItajeZeusCZ8wHaE8&pid=Api',
 'https://tse4.mm.bing.net/th?id=OIP.rvSWtRd_oPRTwDoTCmkP5gHaE8&pid=Api',
 'https://tse3.mm.bing.net/th?id=OIP.NGOmLKzeApsAOK_6oHP0lgHaJ4&pid=Api',
 'https://tse2.mm.bing.net/th?id=OIP.XMsK2_tiu7UxaBddLOzUFgHaLH&pid=Api',
 'https://tse2.mm.bing.net/th?id=OIP.jvjEb4em8aTxsAx78FzfZQHaEK&pid=Api',
 'https://tse3.mm.bing.net/th?id=OIP.whJzSn2dMUxH5dQTZzEyewHaE8&pid=Api',
 'https://tse2.mm.bing.net/th?id=OIP.8xIvwj9-YC-HK96ppjtg7QHaJz&pid=Api',
 'https://tse4.mm.bing.net/th?id=OIP.QBGBwod9GGUxztD6Ar8tKwHaLG&pid=Api',
 'https://tse3.mm.bing.net/th?id=OIP.CFC2G9JinfIvZUcXheEPogHaLG&pid=Api',
 'https://tse3.mm.bing.net/th?id=OIP.O5akhq4Q82ynpnrzkd18_wHaJ4&pid=Api',
 'https://tse4.mm.bing.net/th?id=OIP.S

In [2]:
lik = Path.cwd()/"gdrive/My Drive/colab/Unsplash"
download_urls(lik/"Unplash", links)


##Changing params across multiple searches

In [ ]:
# If you're going to override default params across multiple searches you can use a 
# dictionary like this (so you can change search params for the entire dataset once).

params = {
    "max_results": 12000,             
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.Square,
    
}

duckduckgo_search(lik, "People", "People", **params)
duckduckgo_search(lik,  "Aerial Images", "Aerial Images", **params)
duckduckgo_search(lik,  "Beach", "Beach", **params)
duckduckgo_search(lik,  "Building", "Building", **params)
duckduckgo_search(lik,  "Animal", "Animal", **params)
duckduckgo_search(lik,  "Photography", "Photography", **params)

Duckduckgo search: People


Duckduckgo search: Aerial Images


Exception occured while retrieving https://tse3.mm.bing.net/th?id=OIP.2yqR6S5SstB5xr8xb6N-1wHaH7&pid=Api
Duckduckgo search: Beach


Duckduckgo search: Building


Duckduckgo search: Animal


Duckduckgo search: Photography


[PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/001_2d5d62ff.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/002_fadc8ee5.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/003_d34e034e.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/004_acaaad8e.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/005_80db6457.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/006_fa795821.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/007_744ab8b3.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/008_40ae87f0.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/009_767a2147.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/010_5393f3f7.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/011_d4b2cc11.jpg'),
 PosixPath('/content/gdrive/My Drive/colab/Unsplash/Photography/012_0b76b31e

In [ ]:
# If you're going to override default params across multiple searches you can use a 
# dictionary like this (so you can change search params for the entire dataset once).

params = {
    "max_results": 20000,             
    "img_type":    ImgType.Photo,
    "img_layout":  ImgLayout.Square,
    
}
duckduckgo_search(lik,  "Unsplash", "Unsplash", **params)
duckduckgo_search(lik, "People", "People", **params)
duckduckgo_search(lik,  "Background", "Background", **params)
duckduckgo_search(lik,  "Beach", "Beach", **params)
duckduckgo_search(lik,  "Fall", "Fall", **params)
duckduckgo_search(lik,  "Sunset", "Sunset", **params)
duckduckgo_search(lik,  "Building", "Building", **params)

duckduckgo_search(lik,  "Photography", "Photography", **params)
# duckduckgo_search(root,  "woman", "woman", **params)duckduckgo_search(root,  "face", "human faces", **params)
duckduckgo_search(lik,  "Cloud", "Cloud", **params)
duckduckgo_search(lik,  "Road", "Road", **params)
duckduckgo_search(lik,  "lion", "lion", **params)
duckduckgo_search(lik,  "animal", "animal", **params)
# duckduckgo_search(root,  "priest", "priest", **params)
# duckduckgo_search(root,  "athlete", "athlete", **params)
# duckduckgo_search(root,  "woman", "woman", **params)

Duckduckgo search: Unsplash


Duckduckgo search: People


Duckduckgo search: Background


Duckduckgo search: Beach


Duckduckgo search: Fall


...

Duckduckgo search: Cloud


Duckduckgo search: Road


Duckduckgo search: lion


Exception occured while retrieving https://tse1.explicit.bing.net/th?id=OIP.VQKwm6olNEX3t07QRySJkwHaIu&pid=Api
Duckduckgo search: animal


[PosixPath('/content/https:/unsplash.com/animal/001_86e41826.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/002_12269675.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/003_838d96d4.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/004_1fce0239.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/005_d47a0064.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/006_4845746c.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/007_c1783893.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/008_56e8ac90.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/009_ef4eeb0e.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/010_e939ab7b.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/011_fdb2b2fc.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/012_b0b0e10c.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/013_ca7144bb.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/014_6ffe7af3.jpg'),
 PosixPath('/content/https:/unsplash.com/animal/

##Displaying the image cleaner

In [ ]:
from jmd_imagescraper.imagecleaner import *
display_image_cleaner(lik)

HTML(value='<h2>No images left to display in this folder.</h2>', layout=Layout(visibility='hidden'))

GridBox(children=(VBox(children=(Image(value=b'', layout="Layout(width='150px')"), Button(description='Delete'…